# Detección de Líneas de Carril
## Proyecto Individual - Fundamentos de la Visión por Computador
## Dataset: Udacity Self-Driving Car Nanodegree

**Universidad de Deusto - Facultad de Ingeniería**

---

### Dataset Utilizado

Este proyecto utiliza los videos del dataset público **Udacity Self-Driving Car Nanodegree - Lane Lines Detection**:
- Repositorio: https://github.com/udacity/CarND-LaneLines-P1
- Videos: `solidWhiteRight.mp4`, `solidYellowLeft.mp4`
- Resolución: 960x540 píxeles
- Framerate: 25 FPS

### Técnicas Utilizadas

- Conversión a escala de grises
- Filtrado Gaussiano (reducción de ruido)
- Detección de bordes (algoritmo de Canny)
- Región de interés (ROI)
- Transformada de Hough para detección de líneas
- Suavizado temporal

---

## 1. Importación de Librerías y Configuración

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import time
import os

# Configuración de visualización
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Verificar Dataset de Udacity

Antes de continuar, asegúrate de haber descargado los videos del dataset de Udacity.

In [ ]:
# Videos del dataset Udacity Self-Driving Car
UDACITY_VIDEOS = {
    'solidWhiteRight.mp4': 'https://github.com/udacity/CarND-LaneLines-P1/raw/master/test_videos/solidWhiteRight.mp4',
    'solidYellowLeft.mp4': 'https://github.com/udacity/CarND-LaneLines-P1/raw/master/test_videos/solidYellowLeft.mp4'
}

# Verificar qué videos están disponibles
available_videos = []
for video_name in UDACITY_VIDEOS.keys():
    if os.path.exists(video_name):
        available_videos.append(video_name)
        print(f"✓ Encontrado: {video_name}")
    else:
        print(f"✗ No encontrado: {video_name}")

if not available_videos:
    print("\n" + "="*70)
    print("⚠️  DESCARGA LOS VIDEOS DE UDACITY")
    print("="*70)
    print("\nDescarga al menos uno de estos videos y colócalo en esta carpeta:\n")
    for name, url in UDACITY_VIDEOS.items():
        print(f"  {name}:")
        print(f"  {url}\n")
else:
    VIDEO_PATH = available_videos[0]
    print(f"\n✓ Usando video: {VIDEO_PATH}")

## 3. Clase Principal: LaneDetector

In [ ]:
class LaneDetector:
    """
    Detector de líneas de carril usando técnicas clásicas de visión por computador.
    
    Pipeline:
    1. Conversión a escala de grises
    2. Suavizado Gaussiano
    3. Detección de bordes (Canny)
    4. Región de Interés (ROI)
    5. Transformada de Hough
    6. Separación izquierda/derecha por pendiente
    7. Regresión lineal
    8. Suavizado temporal
    """
    
    def __init__(self, buffer_size=10):
        # Buffers para suavizado temporal
        self.left_lines_buffer = deque(maxlen=buffer_size)
        self.right_lines_buffer = deque(maxlen=buffer_size)
        
        # Parámetros del detector de Canny
        self.canny_low = 50
        self.canny_high = 150
        
        # Parámetros de la transformada de Hough
        self.hough_rho = 2
        self.hough_theta = np.pi/180
        self.hough_threshold = 50
        self.hough_min_line_length = 40
        self.hough_max_line_gap = 100
        
    def process_frame(self, frame):
        """Procesa un frame completo del video."""
        height, width = frame.shape[:2]
        
        # 1. Escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 2. Suavizado Gaussiano
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # 3. Detección de bordes Canny
        edges = cv2.Canny(blurred, self.canny_low, self.canny_high)
        
        # 4. Región de interés (trapecio) - Ajustado para videos Udacity
        mask = np.zeros_like(edges)
        vertices = np.array([[
            (int(width * 0.05), height),
            (int(width * 0.45), int(height * 0.6)),
            (int(width * 0.55), int(height * 0.6)),
            (int(width * 0.95), height)
        ]], dtype=np.int32)
        cv2.fillPoly(mask, vertices, 255)
        roi_edges = cv2.bitwise_and(edges, mask)
        
        # 5. Transformada de Hough
        lines = cv2.HoughLinesP(roi_edges, self.hough_rho, self.hough_theta,
                                self.hough_threshold, 
                                minLineLength=self.hough_min_line_length,
                                maxLineGap=self.hough_max_line_gap)
        
        # 6. Separar líneas izquierda/derecha
        left_lines, right_lines = [], []
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                if x2 - x1 == 0:
                    continue
                slope = (y2 - y1) / (x2 - x1)
                if abs(slope) < 0.5:
                    continue
                if slope < 0 and (x1 + x2) / 2 < width / 2:
                    left_lines.append(line[0])
                elif slope > 0 and (x1 + x2) / 2 > width / 2:
                    right_lines.append(line[0])
        
        # 7. Ajustar líneas por regresión
        left_line = self._fit_line(left_lines, height)
        right_line = self._fit_line(right_lines, height)
        
        # 8. Suavizado temporal
        left_line = self._average_line(left_line, self.left_lines_buffer)
        right_line = self._average_line(right_line, self.right_lines_buffer)
        
        # Dibujar resultado
        result = self._draw_lanes(frame.copy(), left_line, right_line)
        
        return result, edges, roi_edges
    
    def _fit_line(self, lines, img_height):
        if not lines:
            return None
        points_x, points_y = [], []
        for x1, y1, x2, y2 in lines:
            points_x.extend([x1, x2])
            points_y.extend([y1, y2])
        if len(points_x) < 2:
            return None
        try:
            m, b = np.polyfit(points_x, points_y, 1)
            if m == 0:
                return None
            y_bottom, y_top = img_height, int(img_height * 0.6)
            x_bottom = int((y_bottom - b) / m)
            x_top = int((y_top - b) / m)
            return (x_bottom, y_bottom, x_top, y_top)
        except:
            return None
    
    def _average_line(self, current_line, buffer):
        if current_line is not None:
            buffer.append(current_line)
        if not buffer:
            return None
        return tuple(np.mean(buffer, axis=0).astype(int))
    
    def _draw_lanes(self, img, left_line, right_line):
        if left_line:
            cv2.line(img, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (0, 255, 0), 10)
        if right_line:
            cv2.line(img, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (0, 255, 0), 10)
        if left_line and right_line:
            pts = np.array([[left_line[0], left_line[1]], [left_line[2], left_line[3]],
                           [right_line[2], right_line[3]], [right_line[0], right_line[1]]], np.int32)
            overlay = img.copy()
            cv2.fillPoly(overlay, [pts], (0, 255, 0))
            img = cv2.addWeighted(overlay, 0.3, img, 0.7, 0)
        return img
    
    def reset(self):
        self.left_lines_buffer.clear()
        self.right_lines_buffer.clear()

print("✓ Clase LaneDetector cargada")

## 4. Cargar Video de Udacity y Mostrar Frame de Ejemplo

In [ ]:
# Verificar que tenemos un video
if 'VIDEO_PATH' not in dir() or not os.path.exists(VIDEO_PATH):
    raise FileNotFoundError("⚠️ Descarga primero un video de Udacity (ver celda 2)")

# Cargar información del video
cap = cv2.VideoCapture(VIDEO_PATH)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps

# Leer un frame de ejemplo
ret, sample_frame = cap.read()
cap.release()

print("="*50)
print("INFORMACIÓN DEL VIDEO - UDACITY DATASET")
print("="*50)
print(f"  Archivo:     {VIDEO_PATH}")
print(f"  Resolución:  {width}x{height}")
print(f"  FPS:         {fps}")
print(f"  Frames:      {total_frames}")
print(f"  Duración:    {duration:.1f} segundos")
print("="*50)

# Mostrar frame de ejemplo
if ret:
    plt.figure(figsize=(12, 6))
    plt.imshow(cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB))
    plt.title(f'Frame de ejemplo - {VIDEO_PATH} (Udacity Dataset)')
    plt.axis('off')
    plt.show()

## 5. Visualización del Pipeline Completo

In [ ]:
def visualize_pipeline(frame, video_name, save_path='pipeline_visualization.png'):
    """
    Visualiza cada paso del pipeline de detección.
    """
    detector = LaneDetector()
    height, width = frame.shape[:2]
    
    # Ejecutar cada paso del pipeline
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    # ROI
    mask = np.zeros_like(edges)
    vertices = np.array([[
        (int(width * 0.05), height),
        (int(width * 0.45), int(height * 0.6)),
        (int(width * 0.55), int(height * 0.6)),
        (int(width * 0.95), height)
    ]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, 255)
    roi_edges = cv2.bitwise_and(edges, mask)
    
    # Hough lines
    lines = cv2.HoughLinesP(roi_edges, 2, np.pi/180, 50, minLineLength=40, maxLineGap=100)
    hough_img = np.zeros((height, width, 3), dtype=np.uint8)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(hough_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Resultado final
    result, _, _ = detector.process_frame(frame)
    
    # Crear figura
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))
    fig.suptitle(f'Pipeline de Detección de Líneas de Carril - Dataset Udacity ({video_name})', 
                 fontsize=14, fontweight='bold')
    
    # Fila 1
    axes[0,0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    axes[0,0].set_title('1. Imagen Original')
    axes[0,0].axis('off')
    
    axes[0,1].imshow(gray, cmap='gray')
    axes[0,1].set_title('2. Escala de Grises')
    axes[0,1].axis('off')
    
    axes[0,2].imshow(blurred, cmap='gray')
    axes[0,2].set_title('3. Filtro Gaussiano (5x5)')
    axes[0,2].axis('off')
    
    axes[0,3].imshow(edges, cmap='gray')
    axes[0,3].set_title('4. Bordes (Canny 50-150)')
    axes[0,3].axis('off')
    
    # Fila 2
    roi_vis = frame.copy()
    cv2.polylines(roi_vis, vertices, True, (0, 0, 255), 3)
    axes[1,0].imshow(cv2.cvtColor(roi_vis, cv2.COLOR_BGR2RGB))
    axes[1,0].set_title('5. Región de Interés (ROI)')
    axes[1,0].axis('off')
    
    axes[1,1].imshow(roi_edges, cmap='gray')
    axes[1,1].set_title('6. Bordes en ROI')
    axes[1,1].axis('off')
    
    axes[1,2].imshow(cv2.cvtColor(hough_img, cv2.COLOR_BGR2RGB))
    axes[1,2].set_title('7. Transformada de Hough')
    axes[1,2].axis('off')
    
    axes[1,3].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axes[1,3].set_title('8. Resultado Final')
    axes[1,3].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\n✓ Visualización guardada: {save_path}")
    return fig

# Ejecutar visualización con frame del video de Udacity
if ret:
    fig = visualize_pipeline(sample_frame, VIDEO_PATH)

## 6. Análisis de Rendimiento

In [ ]:
def analyze_performance(video_path, max_frames=100):
    """
    Analiza el rendimiento del detector en el video de Udacity.
    """
    detector = LaneDetector()
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: No se puede abrir el video")
        return None
    
    times = []
    detections_left = 0
    detections_right = 0
    total = 0
    
    while total < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        start = time.time()
        detector.process_frame(frame)
        times.append(time.time() - start)
        
        if len(detector.left_lines_buffer) > 0:
            detections_left += 1
        if len(detector.right_lines_buffer) > 0:
            detections_right += 1
        total += 1
    
    cap.release()
    
    return {
        'frames': total,
        'avg_time_ms': np.mean(times) * 1000,
        'std_time_ms': np.std(times) * 1000,
        'fps': 1 / np.mean(times),
        'detection_left': (detections_left / total) * 100,
        'detection_right': (detections_right / total) * 100,
        'times': times
    }

# Ejecutar análisis en el video de Udacity
print(f"Analizando rendimiento en {VIDEO_PATH}...\n")
metrics = analyze_performance(VIDEO_PATH, max_frames=100)

if metrics:
    print("="*55)
    print("MÉTRICAS DE RENDIMIENTO - DATASET UDACITY")
    print("="*55)
    print(f"  Video analizado:          {VIDEO_PATH}")
    print(f"  Frames analizados:        {metrics['frames']}")
    print(f"  Tiempo por frame:         {metrics['avg_time_ms']:.2f} ± {metrics['std_time_ms']:.2f} ms")
    print(f"  FPS de procesamiento:     {metrics['fps']:.1f}")
    print(f"  Detección línea izq:      {metrics['detection_left']:.1f}%")
    print(f"  Detección línea der:      {metrics['detection_right']:.1f}%")
    print("="*55)

In [ ]:
# Visualizar métricas
if metrics:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Métricas de Rendimiento - {VIDEO_PATH} (Udacity)', fontsize=12, fontweight='bold')
    
    # Gráfico de tiempos
    axes[0].plot(metrics['times'], 'b-', alpha=0.7, linewidth=0.8)
    axes[0].axhline(y=np.mean(metrics['times']), color='r', linestyle='--', 
                    label=f'Media: {metrics["avg_time_ms"]:.2f} ms')
    axes[0].set_xlabel('Frame')
    axes[0].set_ylabel('Tiempo (segundos)')
    axes[0].set_title('Tiempo de Procesamiento por Frame')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico de tasas de detección
    categories = ['Línea Izquierda', 'Línea Derecha']
    rates = [metrics['detection_left'], metrics['detection_right']]
    colors = ['#27ae60', '#3498db']
    
    bars = axes[1].bar(categories, rates, color=colors)
    axes[1].set_ylabel('Tasa de Detección (%)')
    axes[1].set_title('Tasa de Detección de Líneas')
    axes[1].set_ylim(0, 105)
    
    for bar, rate in zip(bars, rates):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                    f'{rate:.1f}%', ha='center', fontweight='bold', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('metricas_rendimiento.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Gráficos guardados: metricas_rendimiento.png")

## 7. Procesar Video Completo de Udacity

In [ ]:
def process_video(input_path, output_path):
    """
    Procesa el video completo de Udacity y guarda el resultado.
    """
    detector = LaneDetector()
    cap = cv2.VideoCapture(input_path)
    
    if not cap.isOpened():
        print(f"Error: No se puede abrir {input_path}")
        return False
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Procesando: {input_path}")
    print(f"Video: {width}x{height} @ {fps}fps, {total} frames")
    
    # Codec XVID (más compatible)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    if not writer.isOpened():
        print("Error: No se puede crear el video de salida")
        return False
    
    count = 0
    start_time = time.time()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        result, _, _ = detector.process_frame(frame)
        writer.write(result)
        
        count += 1
        if count % 50 == 0:
            progress = 100 * count / total
            print(f"  Progreso: {count}/{total} frames ({progress:.1f}%)")
    
    elapsed = time.time() - start_time
    
    cap.release()
    writer.release()
    
    print(f"\n✓ Completado: {count} frames en {elapsed:.1f}s")
    print(f"✓ Video guardado: {output_path}")
    return True

# Procesar el video de Udacity
output_video = 'output_' + VIDEO_PATH.replace('.mp4', '.avi')
process_video(VIDEO_PATH, output_video)

## 8. Mostrar Frames de Resultado

In [ ]:
# Mostrar algunos frames del video procesado
if os.path.exists(output_video):
    cap = cv2.VideoCapture(output_video)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Seleccionar 4 frames distribuidos
    frame_indices = [int(total * i / 5) for i in range(1, 5)]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Resultados de Detección - {VIDEO_PATH} (Udacity)', fontsize=14, fontweight='bold')
    
    for idx, (ax, frame_idx) in enumerate(zip(axes.flat, frame_indices)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            ax.set_title(f'Frame {frame_idx}')
            ax.axis('off')
    
    cap.release()
    plt.tight_layout()
    plt.savefig('resultados_deteccion.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Resultados guardados: resultados_deteccion.png")
else:
    print(f"No se encontró el video procesado: {output_video}")

## 9. Resumen y Conclusiones

### Dataset Utilizado

**Udacity Self-Driving Car Nanodegree - Lane Lines Detection**
- Repositorio: https://github.com/udacity/CarND-LaneLines-P1
- Videos: solidWhiteRight.mp4, solidYellowLeft.mp4
- Resolución: 960x540 @ 25 FPS
- Condiciones: Autopista, luz diurna, líneas blancas/amarillas

### Técnicas Implementadas

1. **Conversión a escala de grises**: Reduce complejidad de 3 canales a 1
2. **Filtro Gaussiano (5x5)**: Reduce ruido preservando bordes
3. **Detector de bordes Canny (50-150)**: Umbralización con histéresis
4. **Región de Interés (ROI)**: Trapecio enfocado en el carril
5. **Transformada de Hough**: Detección de segmentos de línea
6. **Clasificación por pendiente**: Separación izquierda/derecha
7. **Regresión lineal**: Ajuste de múltiples segmentos
8. **Suavizado temporal**: Buffer de 10 frames

### Limitaciones Identificadas

- Sensibilidad a cambios bruscos de iluminación
- Dificultades con curvas pronunciadas
- Modelo asume líneas rectas
- Puede fallar con oclusiones

### Posibles Mejoras

- Transformación de perspectiva (bird's eye view)
- Ajuste polinomial de mayor grado para curvas
- Filtro de Kalman para predicción temporal
- Segmentación por color (HSV) para líneas amarillas

In [ ]:
print("\n" + "="*60)
print("PROYECTO COMPLETADO")
print("="*60)
print(f"\nDataset: Udacity Self-Driving Car Nanodegree")
print(f"Video procesado: {VIDEO_PATH}")
print("\nArchivos generados:")
print(f"  - {output_video}: Video con detección de carriles")
print("  - pipeline_visualization.png: Visualización del pipeline")
print("  - metricas_rendimiento.png: Gráficos de métricas")
print("  - resultados_deteccion.png: Frames de ejemplo")
print("\n" + "="*60)